In [1]:
from PIL import Image
import torch
import pandas as pd
import os

In [ ]:
class load_ds(torch.utils.data.Dataset):
    def __init__(self, csv, images_dir, label_dir, S, B, C, transform=None):
        self.associations = pd.read_csv(csv)
        self.img_dir      = images_dir
        self.lbl_dir      = label_dir
        self.transform    = transform
        self.S            = S
        self.B            = B
        self.C            = C
        
    def __len__(self):
        return len(self.associations)
        
    def __getitem__(self, index):
        img_path   = os.path.join(self.img_dir, self.associations.iloc[index, 0])
        label_path = os.path.join(self.lbl_dir, self.associations.iloc[index, 1])
        bboxes     = []
        
        with open(label_path) as x:
            for i in x.readlines():
                class_label, x, y, w, h = [float(x) if float(x) != int(float(x)) else int(x)
                                            for x in i.replace("\n", "").split()]
                
                bboxes.append([class_label, x, y, w, h])
        
        img = Image.open(img_path)
        if self.transform:
            for i in self.transform:
                img, bboxes = i(img), bboxes
                
        
        labels = torch.zeros((self.S, self.S, self.C + self.B*5))
        for box in bboxes:
            class_label, x, y, w, h = box
            class_label = int(class_label)

            # i,j represents the cell row and cell column
            i, j = int(self.S * y), int(self.S * x)
            x_cell, y_cell = self.S * x - j, self.S * y - i
            w_cell, h_cell = (w*self.S, h*self.S)
            
            
            if labels[i, j, 20] == 0:
                # Set that there exists an object
                labels[i, j, 20] = 1

                # Box coordinates
                box_coords = torch.tensor([x_cell, y_cell, w_cell, h_cell])
                labels[i, j, 21:25] = box_coords

                # Set one hot encoding for class_label
                labels[i, j, class_label] = 1

        return img, labels
                        